# Метод первый - Явное нахождение название темы в словах
###### когда никак не получается найти датасет

### Будем парсить страничку, оставляя только слова и глаголы и в них искать слова из данных тематик
#### А выводить будем [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF) значения в процентах

In [1]:
# данные тематики
t = """/Наука и образование
    /Наука и образование/наука
    /Наука и образование/наука/математика
    /Наука и образование/наука/физика
    /Наука и образование/наука/химия
    /Наука и образование/наука/информатика
    /Наука и образование/наука/информатика/биоинформатика
    /Наука и образование/наука/информатика/анализ данных
    /Наука и образование/наука/литература
    /Наука и образование/образование
    /Наука и образование/образование/школьное
    /Наука и образование/образование/высшее
    /Наука и образование/образование/дополнительное
    /Наука и образование/образование/дополнительное/GoTo
    /Политика
    /Политика/Внутренняя
    /Политика/Внешняя
    /Экономика и бизнес
    /Экономика и бизнес/Бизнес
    /Экономика и бизнес/Бизнес/Стартапы
    /Экономика и бизнес/Бизнес/Стартапы/E-Contenta
    /Экономика и бизнес/Бизнес/Крупные компании
    /Экономика и бизнес/Экономика
    /Отдых и развлечения
    /Отдых и развлечения/Кино
    /Отдых и развлечения/Театр
    /Отдых и развлечения/Компьютерные игры
    /Здоровье и красота/Фитнес
    /Здоровье и красота/Медицина
    /Здоровье и красота/Косметология""".split("\n")

# массив последних тем в иерархии (топики) и словарь, с пом. которого можно восстановить полную структуру
topics = [x[1:].split("/")[-1] for x in t]
restore = {topics[i]: t[i] for i in range(len(topics))}

In [2]:
topics

['Наука и образование',
 'наука',
 'математика',
 'физика',
 'химия',
 'информатика',
 'биоинформатика',
 'анализ данных',
 'литература',
 'образование',
 'школьное',
 'высшее',
 'дополнительное',
 'GoTo',
 'Политика',
 'Внутренняя',
 'Внешняя',
 'Экономика и бизнес',
 'Бизнес',
 'Стартапы',
 'E-Contenta',
 'Крупные компании',
 'Экономика',
 'Отдых и развлечения',
 'Кино',
 'Театр',
 'Компьютерные игры',
 'Фитнес',
 'Медицина',
 'Косметология']

In [3]:
import requests
import re
import nltk
from html2text import HTML2Text
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

import pymorphy2

### Firstly, we need to download the page we are looking for

In [4]:
web_url = "https://ru.wikipedia.org/wiki/%D0%9E%D0%BF%D1%80%D0%B5%D0%B4%D0%B5%D0%BB%D0%B8%D1%82%D0%B5%D0%BB%D1%8C_%D0%92%D0%B0%D0%BD%D0%B4%D0%B5%D1%80%D0%BC%D0%BE%D0%BD%D0%B4%D0%B0"
page = requests.get(web_url)
soup = BeautifulSoup(page.content, "lxml")

### Next step we clear text from punctuation, html tags and apply regular expression

In [5]:
h = HTML2Text()
h.ignore_links = True

In [6]:
raw_text = h.handle(soup.body.text)

In [7]:
only_russian_text = re.sub("[^а-яА-Я]", " ", raw_text).strip()

### Now define a function

In [8]:
def get_nouns_list_from_url(web_url):
    # получим текст
    page = requests.get(web_url)
    soup = BeautifulSoup(page.content, "lxml")
    h = HTML2Text()
    h.ignore_links = True
    raw_text = h.handle(soup.body.text)
    only_russian_text = re.sub("[^а-яА-Я]", " ", raw_text).strip()
    
    # оставим только слова без стоп-слов длины [3, 15] и только существительные и глаголы в нормальной форме
    morph = pymorphy2.MorphAnalyzer()
    words = only_russian_text.split()
    stoplist = stopwords.words('russian')
    data = []
    for word in words:
        if 2 < len(word) < 16 and word not in stoplist:
            p = morph.parse(word)[0]
            if p.tag.POS == 'NOUN' or "VERB":
                data.append(str(p.normal_form))
    
    return data

In [9]:
get_nouns_list_from_url(web_url)

['определитель',
 'вандермонд',
 'материал',
 'википедия',
 'свободный',
 'энциклопедия',
 'перейти',
 'навигация',
 'поиск',
 'определитель',
 'вандермонд',
 'называться',
 'определитель',
 'существовать',
 'равно',
 'степень',
 'иной',
 'слово',
 'делиться',
 'различный',
 'многочлен',
 'степень',
 'значит',
 'равный',
 'произведение',
 'точность',
 'константа',
 'убедиться',
 'раскрыть',
 'скобка',
 'константа',
 'равный',
 'единица',
 'дать',
 'формула',
 'показывать',
 'определитель',
 'вандермонд',
 'равный',
 'нуль',
 'существовать',
 'хотя',
 'один',
 'пара',
 'такой',
 'определитель',
 'вандермонд',
 'иметь',
 'многочисленный',
 'применение',
 'разный',
 'область',
 'математика',
 'например',
 'решение',
 'задача',
 'интерполяция',
 'многочлен',
 'задача',
 'нахождение',
 'многочлен',
 'степень',
 'график',
 'который',
 'проходить',
 'задать',
 'точка',
 'плоскость',
 'абсцисса',
 'определитель',
 'вандермонд',
 'возникать',
 'определитель',
 'система',
 'линейный',
 'уравнени

### Now we need to get top tf-idf values and show corresponding words

In [12]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# инициализируем модельки
count_vect = CountVectorizer()
tf_idf = TfidfTransformer()

In [16]:
# применяем CountVectorizer к строке из слов, вытянутых с веб-страницы
X = count_vect.fit_transform([" ".join(get_nouns_list_from_url(web_url))])
X = tf_idf.fit_transform(X)

In [17]:
# отсортированный по невозрастанию массив пар индекс-tfidf значение
tfidf_repr = sorted([(a[0][1], a[1]) for a in X.todok().items()], key=lambda a: a[1], reverse=True)

In [18]:
tfidf_repr[:10]

[(75, 0.32551538350846376),
 (3, 0.32551538350846376),
 (149, 0.28482596056990578),
 (92, 0.28482596056990578),
 (1, 0.20344711469278987),
 (52, 0.20344711469278987),
 (47, 0.20344711469278987),
 (127, 0.16275769175423188),
 (55, 0.16275769175423188),
 (133, 0.16275769175423188)]

In [19]:
# теперь сделаем из этого словарь
tfidf_dict = {id_ : tfidf for id_, tfidf in tfidf_repr}

In [20]:
# словарь для доставания слов из CountVectorizer словаря - то, что имеем
good_vocab = {}
for word, _id in count_vect.vocabulary_.items():
    good_vocab[str(_id)] = word

In [21]:
# хочется, чтобы сумма tf-idf значений в процентах была 100 :)
def softmax(X):
    a = np.array(X)
    multiplier = 100 / np.sum(a)
    return list(a * multiplier)

In [22]:
res = []
a = []
b = []
for k in topics:
    # сначала название топика приведем к нормальной форме, чтобы лучше найти его в count_vect
    word = morph.parse(k)[0].normal_form
    _id = count_vect.vocabulary_.get(word)
    if _id:
        # по слову - айдишник, по айдишнику - tfidf значение. Если оно существует, то добавляем топик и tfidf в соотв. массивы
        tfidf = tfidf_dict.get(_id)
        if tfidf:
            a.append(restore[k])
            b.append(tfidf)
b_soft = softmax(b)
# только сейчас добавляем в результирующий массив изначальное название топика и его процентное содержание
for i in range(len(a)):
    res.append("{} — {:.2f}%".format(a[i], b_soft[i]))
    
res = sorted(res, key=lambda x: float(x.split(" — ")[1][:-1]), reverse=True)

In [23]:
res[:10]

['    /Наука и образование/наука — 28.57%',
 '    /Наука и образование/наука/математика — 14.29%',
 '    /Наука и образование/наука/литература — 14.29%',
 '    /Наука и образование/образование/высшее — 14.29%',
 '    /Наука и образование/образование/дополнительное — 14.29%',
 '    /Политика — 14.29%']

### Now let's create all-in-one function (besause we can :) )

In [24]:
import requests
import re
import nltk
import pymorphy2
import numpy as np
from html2text import HTML2Text
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer


def method1(web_url, topn=10):
    t = """/Наука и образование
    /Наука и образование/наука
    /Наука и образование/наука/математика
    /Наука и образование/наука/физика
    /Наука и образование/наука/химия
    /Наука и образование/наука/информатика
    /Наука и образование/наука/информатика/биоинформатика
    /Наука и образование/наука/информатика/анализ данных
    /Наука и образование/наука/литература
    /Наука и образование/образование
    /Наука и образование/образование/школьное
    /Наука и образование/образование/высшее
    /Наука и образование/образование/дополнительное
    /Наука и образование/образование/дополнительное/GoTo
    /Политика
    /Политика/Внутренняя
    /Политика/Внешняя
    /Экономика и бизнес
    /Экономика и бизнес/Бизнес
    /Экономика и бизнес/Бизнес/Стартапы
    /Экономика и бизнес/Бизнес/Стартапы/E-Contenta
    /Экономика и бизнес/Бизнес/Крупные компании
    /Экономика и бизнес/Экономика
    /Отдых и развлечения
    /Отдых и развлечения/Кино
    /Отдых и развлечения/Театр
    /Отдых и развлечения/Компьютерные игры
    /Здоровье и красота/Фитнес
    /Здоровье и красота/Медицина
    /Здоровье и красота/Косметология""".split("\n")
    topics = [x[1:].split("/")[-1] for x in t]
    restore = {topics[i]: t[i] for i in range(len(topics))}
    
    
    # получим текст
    page = requests.get(web_url)
    soup = BeautifulSoup(page.content, "lxml")
    h = HTML2Text()
    h.ignore_links = True
    raw_text = h.handle(soup.body.text)
    only_russian_text = re.sub("[^а-яА-Я]", " ", raw_text).strip()
    
    # оставим только слова без стоп-слов длины [3, 15] и только существительные и глаголы в нормальной форме
    morph = pymorphy2.MorphAnalyzer()
    words = only_russian_text.split()
    stoplist = stopwords.words('russian')
    data = []
    for word in words:
        if 2 < len(word) < 16 and word not in stoplist:
            p = morph.parse(word)[0]
            if p.tag.POS == 'NOUN' or "VERB":
                data.append(str(p.normal_form))
                
    count_vect = CountVectorizer()
    tf_idf = TfidfTransformer()
    X = count_vect.fit_transform([" ".join(data)])
    X = tf_idf.fit_transform(X)
    
    tfidf_repr = sorted([(a[0][1], a[1]) for a in X.todok().items()], key=lambda a: a[1], reverse=True)
    tfidf_dict = {id_ : tfidf for id_, tfidf in tfidf_repr}
    
    good_vocab = {str(_id) : word for word, _id in count_vect.vocabulary_.items()}
    
    res = []
    a = []
    b = []
    for k in topics:
        # сначала название топика приведем к нормальной форме, чтобы лучше найти его в count_vect
        word = morph.parse(k)[0].normal_form
        _id = count_vect.vocabulary_.get(word)
        if _id:
            # по слову - айдишник, по айдишнику - tfidf значение. Если оно существует, то добавляем топик и tfidf в соотв. массивы
            tfidf = tfidf_dict.get(_id)
            if tfidf:
                a.append(restore[k])
                b.append(tfidf)
                
    v = np.array(b)
    if np.sum(v) == 0:
        print("Вероятно, вы показываете мне английский текст, я так не умею")
        print("Ну или вы нашли сайт, который не подходит ни под одну категорию")
        return []
    multiplier = 100 / np.sum(v)
    b_soft = list(v * multiplier)

    # только сейчас добавляем в результирующий массив изначальное название топика и его процентное содержание
    for i in range(len(a)):
        res.append("{} — {:.2f}%".format(a[i], b_soft[i]))
        
    return sorted(res, key=lambda x: float(x.split(" — ")[1][:-1]), reverse=True)[:topn]

In [25]:
method1("http://goto.msk.ru/hackathon/")

['    /Наука и образование/образование — 31.58%',
 '    /Наука и образование/наука — 15.79%',
 '    /Здоровье и красота/Медицина — 15.79%',
 '    /Экономика и бизнес/Бизнес — 10.53%',
 '    /Наука и образование/образование/школьное — 5.26%',
 '    /Экономика и бизнес/Экономика — 5.26%',
 '    /Отдых и развлечения/Кино — 5.26%',
 '    /Отдых и развлечения/Театр — 5.26%',
 '    /Здоровье и красота/Фитнес — 5.26%']

In [26]:
method1("https://www.kinopoisk.ru/film/648440/")

['    /Отдых и развлечения/Кино — 83.33%',
 '    /Политика/Внутренняя — 11.11%',
 '    /Наука и образование/образование/высшее — 5.56%']

In [27]:
method1("https://e-contenta.com/ru/")

['    /Экономика и бизнес/Бизнес — 75.00%',
 '    /Наука и образование/образование/высшее — 25.00%']

In [28]:
method1("https://ru.wikipedia.org/wiki/C%2B%2B")

['    /Наука и образование/образование/высшее — 56.41%',
 '    /Наука и образование/образование/дополнительное — 10.26%',
 '    /Наука и образование/наука/математика — 7.69%',
 '    /Наука и образование/наука/литература — 7.69%',
 '    /Политика/Внешняя — 7.69%',
 '    /Наука и образование/наука/информатика — 5.13%',
 '    /Политика — 2.56%',
 '    /Политика/Внутренняя — 2.56%']